# 1. Load the images and the labels

In [1]:
import os
import numpy as np
import cv2

In [3]:
DATASET_PATH = '../data/Fish_Dataset/Fish_Dataset/'

folders = os.listdir(DATASET_PATH)

# Get the labels name
labels = []
for f in folders:
    if not('.m' in f) and not('.txt' in f):
        labels.append(f)
labels = np.array(labels)


# Get the images for each label
img_shape = (128, 128) #resizing images to this shape ( after converting color image to gray image)
n_sample_per_class = 600 #Keep only this number of samples per class (expensive complexity issue)
nb_img = n_sample_per_class*len(labels) #1000 images per class

X = np.zeros((nb_img, img_shape[0]*img_shape[1])) # feature matrix
Y = np.zeros(nb_img) #labels vector
for i in range(len(labels)):
    folder = os.path.join(DATASET_PATH, labels[i], labels[i])
    images = os.listdir(folder)
    np.random.shuffle(images)
    for j in range(n_sample_per_class):
        img_path = os.path.join(folder, images[j])
        img = cv2.imread(img_path, 0) #Load the gray image
        #Resizing
        img = cv2.resize(src=img, dsize=img_shape)
        #Normalization
        img = img/255
        #Add the image in the feature matrix and its labels to the label vector
        X[i*n_sample_per_class+j, :] = img.flatten()
        Y[i*n_sample_per_class+j] = np.argwhere(labels==labels[i])[0]

FileNotFoundError: [Errno 2] No such file or directory: '../data/Fish_Dataset/Fish_Dataset/'

In [ ]:
print("Feature matrix : \n", X)
print("Label vector : \n", Y)

# 2. Use the PCA technique for reduction of dimentionality

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def pca_per_class(X, n_components):
    """
    Create another dataset by selecting the n_components the most correlated for each class by using PCA for each class
    :param X: the inital matrix feature
    :param n_components: number of components to keep for each class
    :return: X_pca : the new matrix feature
    """
    pca_classes = [] #list with the PCA instance for each class
    for i in range(len(labels)):
        idx_samples = np.argwhere(Y == i)[:, 0]
        X_class_i = X[idx_samples, :] # keep only the samples for the class i
        #Instanciate the PCA
        pca_i = PCA(n_components=n_components)
        #Train the PCA
        pca_i.fit_transform(X_class_i)
        #Add to the list
        pca_classes.append(pca_i)
    #Transform the inital matrix feature with the PCA of the first class
    X_pca = pca_classes[0].transform(X)
    #Concatenate the other transform matrix to create the new matrix feature
    for i in range(1, len(labels)):
        X_pca_i = pca_classes[i].transform(X)
        X_pca = np.concatenate((X_pca, X_pca_i), axis=1)
    return X_pca

X_pca_10 = pca_per_class(X, 10)

# 3. Running classifier models

Testing 3 different classifier models:
- SVM
- Nearest neighbour
- Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca_10, Y, test_size=0.2, random_state=0)

## 3.1 SVM classifier model

In [ ]:
from sklearn import svm

def run_svm_model(X_train, y_train, X_test):
    model = svm.SVC(random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model, y_pred

model, y_pred = run_svm_model(X_train, y_train, X_test)

## 3.2 Nearest Neighbour classifier model

In [ ]:
from sklearn import neighbors

def run_nn_model(X_train, y_train, X_test):
    model = neighbors.KNeighborsClassifier(n_neighbors=3)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model, y_pred

model, y_pred = run_nn_model(X_train, y_train, X_test)
type(model)

## 3.3 Decision tree model

In [ ]:
from sklearn import tree

def train_tree_model(X_train, y_train, X_test):
    model = tree.DecisionTreeClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model, y_pred

model, y_pred = train_tree_model(X_train, y_train, X_test)

# 4. Evaluate the model via cross validation

Evaluations:
1. Classification report
2. Main metrics (accuracy, precision, recall, F1 score)
3. Confusion matrix
4. ROC (TPR, FPR)

## 4.1 Classification report

In [ ]:
from sklearn.metrics import classification_report as sickit_classification_report

def classification_report(model, y_test, y_pred):
    print(f'Classification report for model {model}')
    print(sickit_classification_report(y_test, y_pred))

classification_report(svm_model, y_test, svm_pred)
classification_report(nn_model, y_test, nn_pred)
classification_report(tree_model, y_test, tree_pred)

## 4.2. Evaluating the main metrics (accuracy, precision, recall, F1 score)

In [ ]:
# Scores
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Cross Validation
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
# Plots
import matplotlib.pyplot as plt

In [ ]:
def plot_metrics_evolution_cv(score_cv, metric_keys, colors, title, ax=None):
    if ax is None:
        plt.figure()
    for i in range(len(metric_keys)):
        label = metric_keys[i].replace('test_', '')
        if ax is None :
            plt.plot(score_cv[metric_keys[i]], color=colors[i], label=label, linewidth=1)
            plt.plot(np.arange(0, len(score_cv[metric_keys[i]])), np.mean(score_cv[metric_keys[i]])*np.ones(len(score_cv[metric_keys[i]])), linestyle='dashed', color=colors[i])
            print('Mean of the {} : {}'.format(label, np.round(np.mean(score_cv[metric_keys[i]]), decimals=3)))
        else:
            ax.plot(score_cv[metric_keys[i]], color=colors[i], label=label, linewidth=1)
            ax.plot(np.arange(0, len(score_cv[metric_keys[i]])), np.mean(score_cv[metric_keys[i]])*np.ones(len(score_cv[metric_keys[i]])), linestyle='dashed', color=colors[i])
    plt.xlabel('Fold')
    plt.ylabel('Value')
    plt.title(title)
    plt.legend()
    plt.show()


def main_metrics(model, X, Y):
    scoring = {
        "Accuracy": make_scorer(accuracy_score),
        "Precision": make_scorer(precision_score, average='macro'),
        'Recall': make_scorer(recall_score, average='macro'),
        'F1_score': make_scorer(f1_score, average='macro')
    }
    scores = cross_validate(model, X, Y, scoring=scoring, cv=10)
    predictions = cross_val_predict(model, X, Y, cv=10)
    colors = ['k', 'b', 'g', 'r']
    metric_keys = ['test_Accuracy', 'test_Precision', 'test_Recall', 'test_F1_score']
    plot_metrics_evolution_cv(scores, metric_keys, colors, title=f'Metrics values during 10-fold cv for model {model}')


main_metrics(model, X_pca_10, Y)

The plot shows the values of the metrics in each fold of the cross-validation. We can see that there are some fluctuations of the metrics depending on the fold. Compared to the result with the train_test_method (metrics evaluation in fold 0), the cross-validation results show these fluctuations and the means of the metrics give a more realistic estimation of them.

## 4.3 Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix as sickit_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
def confusion_matrix(model, y_test, y_pred):
    labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    confMat = sickit_confusion_matrix(y_test, y_pred, labels=labels)
    confMat_display = ConfusionMatrixDisplay(confusion_matrix=confMat, display_labels=labels)
    confMat_display.plot()
    confMat_display.ax_.set_title(f'Confusion Matrix for model {model}')

confusion_matrix(model, y_test, y_pred)

We can see on the confusion matrix that the classifier (here DecisionTree) have better results in some classes compared to the other. For example, the classes 3 and 6 have greater results than the classes 4 and 5.

## 4.4 Receiver Operating Characteristic (ROC)

In [ ]:
from sklearn.metrics import roc_curve, auc
from itertools import cycle

In [ ]:
def plot_roc(model, roc_auc, n_classes, fpr, tpr):
    colors = cycle(["black", "blueviolet", "cornflowerblue", "aqua", "lime", "palegoldenrod", "lightsalmon", "orangered", "pink"])
    plt.figure(figsize=(8, 6))
    for i, color in zip(range(n_classes), colors):
        plt.plot(
            fpr[i],
            tpr[i],
            color=color,
            lw=2,
            label="class:{0}, area:{1:0.2f})".format(i, roc_auc[i]),
        )
    plt.plot([0, 1], [0, 1], "k--", lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC evaluation for model {model}")
    plt.legend(loc="lower right", prop={'size': 7})
    plt.show()


def roc(model, y_test, y_pred):
    n_classes = 9
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], thresholds = roc_curve(y_test, y_pred, pos_label=i)
        roc_auc[i] = auc(fpr[i], tpr[i])
    plot_roc(model, roc_auc, n_classes, fpr, tpr)


roc(model, y_test, y_pred)

As we have seen with the confusion matrix, some classes have poor results while other seems to be good classified.

## 4.4 Box plots

In [ ]:
SCORING = {
    "Accuracy": make_scorer(accuracy_score),
    "Precision": make_scorer(precision_score, average='macro'),
    'Recall': make_scorer(recall_score, average='macro'),
    'F1_score': make_scorer(f1_score, average='macro')
}
KEYS = ['test_Accuracy', 'test_Precision', 'test_Recall', 'test_F1_score']

def plot_scores(scores, keys, labels):

    plt.figure(figsize=(10, 10))
    for i in range(len(keys)):
        plt.subplot(2, 2, i+1)
        metric_key = keys[i]
        plt.title('{} in cross-validation per dataset'.format(metric_key.split('test_')[-1]))
        plt.boxplot([score[metric_key] for score in scores])
        plt.xticks(np.arange(1, len(labels) + 1), labels, rotation=90)
        plt.xlabel('Dataset')
        plt.ylabel('Value')
    plt.tight_layout()

### 4.4.1 (evaluating the PCAS)
Compare the metrics results on different datasets (2, 5 and 10 top features per class and 216 top features)

In [ ]:
def evaluate_pcas(model):
    x_pcas = []
    x_pcas.append(pca_per_class(X, 2))
    x_pcas.append(pca_per_class(X, 5))
    x_pcas.append(pca_per_class(X, 10))
    pca_216 = PCA(216)
    tr_pca_216 = pca_216.fit_transform(X)
    x_pcas.append(tr_pca_216)
    scores = []
    for x_pca in x_pcas:
        if isinstance(model, svm.SVC):
            new_model = svm.SVC(random_state=0)
        elif isinstance(model, neighbors._classification.KNeighborsClassifier):
            new_model = neighbors.KNeighborsClassifier(n_neighbors=3)
        else:
            new_model = tree.DecisionTreeClassifier()
        scores.append(cross_validate(new_model, x_pca, Y, scoring=SCORING, cv=10))
    return scores

In [ ]:
scores = evaluate_pcas(tree.DecisionTreeClassifier())
plot_scores(scores, KEYS, ['2 TF/class', '5 TF/class', '10 TF/class', '216 TF'])

### 4.4.2 Evaluate the models
Compare the metrics results of different models trained and evaluated (with cross-validation) on the 10 top features per class dataset

In [ ]:
def evaluate_models(X, Y):
    models = [
        svm.SVC(random_state=0),
        neighbors.KNeighborsClassifier(n_neighbors=3),
        tree.DecisionTreeClassifier()
    ]
    scores = []
    for model in models:
        scores.append(cross_validate(model, X, Y, scoring=SCORING, cv=10))
    return scores

In [ ]:
scores = evaluate_models(X_pca_10, Y)
plot_scores(scores, KEYS, ['SVC', 'Nearest neighbour', 'Decision tree'])